# Train OCR text Detector quick example

In [2]:
import os
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import sys
import glob
import torch
import warnings
from datetime import datetime
from matplotlib import pyplot as plt
from torch import nn
from torchvision.models import shufflenet_v2_x2_0

warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../../../../')
sys.path.append(NOMEROFF_NET_DIR)

from nomeroff_net.data_loaders import TextImageGenerator
from nomeroff_net.pipes.number_plate_text_readers.base.ocr import OCR

In [4]:
plt.rcParams["figure.figsize"] = (10, 10)

In [5]:
%matplotlib inline 

In [6]:
# auto download latest dataset
from nomeroff_net.tools import modelhub

# auto download latest dataset
info = modelhub.download_dataset_for_model("kg")
PATH_TO_DATASET = info["dataset_path"]

model_info = modelhub.download_model_by_name("kg")
MODEL_PATH = model_info["path"]

In [7]:
kg = OCR(model_name="kg", letters=model_info["letters"],
         linear_size=model_info["linear_size"], max_text_len=model_info["max_text_len"],
         height=model_info["height"], width=model_info["width"],
         color_channels=model_info["color_channels"],
         hidden_size=model_info["hidden_size"], backbone=model_info["backbone"])
kg.prepare(PATH_TO_DATASET, use_aug=False, num_workers=1)
kg.load()

GET ALPHABET
Max plate length in "val": 8
Max plate length in "train": 8
Max plate length in "test": 8
Letters train  {'5', 'J', 'E', 'A', 'T', '4', '2', 'W', 'L', 'G', 'F', 'P', 'O', 'Z', 'M', 'K', '6', 'U', 'V', 'I', '8', '3', 'N', 'H', 'X', 'Q', 'R', '9', '1', '0', 'S', 'D', 'Y', 'B', '7', 'C'}
Letters val  {'5', 'J', 'E', 'A', 'T', '4', '2', 'L', 'W', 'G', 'F', 'O', 'P', 'Z', 'M', 'K', '6', 'U', 'V', 'I', '8', '3', 'N', 'H', 'X', 'Q', 'R', '9', '1', '0', 'S', 'D', 'Y', 'B', '7', 'C'}
Letters test  {'5', 'J', 'E', 'A', 'T', '4', '2', 'W', 'G', 'L', 'F', 'P', 'O', 'Z', 'M', 'K', '6', 'U', 'V', 'I', '8', '3', 'N', 'H', 'X', 'Q', 'R', '9', '1', '0', 'S', 'D', 'Y', 'B', '7', 'C'}
Max plate length in train, test and val do match
Letters in train, val and test do match
Letters: 0 1 2 3 4 5 6 7 8 9 A B C D E F G H I J K L M N O P Q R S T U V W X Y Z
START BUILD DATA


100%|██████████| 1068/1068 [00:00<00:00, 19409.66it/s]


DATA PREPARED


NPOcrNet(
  (conv_nn): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [8]:
kg.test_acc(verbose=True)


[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/kg/autoriaNumberplateOcrKg-2020-12-31/test/img/14248352_1_2.png
Predicted: o2809e 			 True: d2809e

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/kg/autoriaNumberplateOcrKg-2020-12-31/test/img/8278291_0_2.png
Predicted: e6944d 			 True: e5944d

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/kg/autoriaNumberplateOcrKg-2020-12-31/test/img/kg4632_0_2.png
Predicted: b5648ah 			 True: b5648ai

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/kg/autoriaNumberplateOcrKg-2020-12-31/test/img/12766622_0_2.png
Predicted: 06193aaz 			 True: 06193abz

[INFO] /mnt/data/var/www/nomeroff-net/nomeroff_net/tools/../../data/./dataset/TextDetector/kg/autoriaNumberplateOcrKg-2020-12-31/test/img/15759009_2_2.png
Predicted: 08810abk 			 True: 08810ack

[INFO] /mnt/data/var/www/nomeroff-net

0.9756554307116105

In [9]:
for dataset_by_np_format in glob.glob(os.path.join(info["dataset_path"], "test_by_np_format/*")):
    print(os.path.basename(dataset_by_np_format))
    image_generator = TextImageGenerator(
            dataset_by_np_format,
            kg.letters,
            kg.max_text_len,
            img_w=kg.width,
            img_h=kg.height,
            batch_size=kg.batch_size,
            with_aug=False)
    acc = kg.acc_calc(image_generator, verbose=False)
    print('Validaton Accuracy: ', acc, "in", len(image_generator))

###@@


100%|██████████| 1068/1068 [00:00<00:00, 9940.01it/s]


Validaton Accuracy:  1.0 in 11
#####@@


100%|██████████| 1068/1068 [00:00<00:00, 7856.11it/s]


Validaton Accuracy:  1.0 in 15
@####@@


...

Validaton Accuracy:  0.9757575757575757 in 330
@###@@@


100%|██████████| 1068/1068 [00:00<00:00, 36210.70it/s]


Validaton Accuracy:  0.5 in 2
####@@


100%|██████████| 1068/1068 [00:00<00:00, 2189.31it/s]


Validaton Accuracy:  0.9857142857142858 in 70
